In [3]:
import ee
import os
import json

def dapatkan_token_baru():
    print("="*60)
    print("GENERATOR TOKEN GEE UNTUK STREAMLIT (MODE NO-GCP)")
    print("="*60)
    print("1. Browser akan terbuka. Login dengan akun Google Anda.")
    print("2. Jika diminta Project, pilih 'Use existing' atau default.")
    print("3. Setelah sukses, token akan muncul di sini.")
    print("-" * 60)
    
    try:
        # Memaksa mode notebook agar tidak perlu gcloud CLI
        # Ini akan memunculkan kode otentikasi
        ee.Authenticate(auth_mode='notebook')
        
        # Cari file kredensial yang baru saja dibuat
        home_dir = os.path.expanduser("~")
        cred_path = os.path.join(home_dir, '.config', 'earthengine', 'credentials')
        
        if os.path.exists(cred_path):
            with open(cred_path) as f:
                data = json.load(f)
                token = data.get('refresh_token')
                
            print("\n" + "#"*60)
            print("BERHASIL! INI DATA RAHASIA ANDA:")
            print("#"*60)
            print(f"\nGEE_REFRESH_TOKEN = {token}")
            print("\n" + "#"*60)
            print("PENTING: ")
            print("1. Copy token panjang di atas.")
            print("2. Masukkan ke Streamlit Secrets.")
            print("3. KOSONGKAN/HAPUS 'GEE_CLIENT_ID' dan 'GEE_CLIENT_SECRET' di Streamlit.")
            print("   (Kita akan pakai ID default GEE di script app.py nanti)")
            print("#"*60)
        else:
            print("Gagal menemukan file token. Coba jalankan ulang.")
            
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    dapatkan_token_baru()

GENERATOR TOKEN GEE UNTUK STREAMLIT (MODE NO-GCP)
1. Browser akan terbuka. Login dengan akun Google Anda.
2. Jika diminta Project, pilih 'Use existing' atau default.
3. Setelah sukses, token akan muncul di sini.
------------------------------------------------------------

############################################################
BERHASIL! INI DATA RAHASIA ANDA:
############################################################

GEE_REFRESH_TOKEN = <REFRESH_TOKEN>

############################################################
PENTING: 
1. Copy token panjang di atas.
2. Masukkan ke Streamlit Secrets.
3. KOSONGKAN/HAPUS 'GEE_CLIENT_ID' dan 'GEE_CLIENT_SECRET' di Streamlit.
   (Kita akan pakai ID default GEE di script app.py nanti)
############################################################


In [ ]:
# =========================================================
# RIAU FIRE COMMAND CENTER (RFCC)
# app.py — FINAL, BERSIH, & CLOUD-SAFE
# =========================================================

import streamlit as st
import pandas as pd
import numpy as np
import os
import pydeck as pdk
import shapely.wkt
import shapely.geometry
import ee
import altair as alt
from datetime import datetime, timedelta
import gdown
from google.oauth2.credentials import Credentials

# =========================================================
# 1️⃣ KONFIGURASI HALAMAN
# =========================================================
st.set_page_config(
    layout="wide",
    page_title="Riau Fire Command Center (RFCC)",
    page_icon="🔥",
    initial_sidebar_state="expanded"
)

# =========================================================
# 2️⃣ AUTENTIKASI GOOGLE EARTH ENGINE (WAJIB & FINAL)
# =========================================================
def init_gee():
    try:
        refresh_token = st.secrets["GEE_REFRESH_TOKEN"]

        DEFAULT_CLIENT_ID = "517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com"
        DEFAULT_CLIENT_SECRET = "secret"

        creds = Credentials(
            None,
            refresh_token=refresh_token,
            token_uri="https://oauth2.googleapis.com/token",
            client_id=DEFAULT_CLIENT_ID,
            client_secret=DEFAULT_CLIENT_SECRET
        )

        ee.Initialize(credentials=creds)

    except Exception as e:
        st.error("❌ GAGAL AUTENTIKASI GOOGLE EARTH ENGINE")
        st.exception(e)
        st.stop()

init_gee()

# =========================================================
# 3️⃣ STYLE UI
# =========================================================
st.markdown("""
<style>
.stApp { background-color: #0E1117; color: #E0E0E0; }
div[data-testid="stMetric"] {
    background-color: #1A1C24;
    border-left: 5px solid #FF4B2B;
    padding: 15px;
    border-radius: 8px;
}
h1 {
    background: linear-gradient(to right, #FF4B2B, #FF416C);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-weight: 800;
}
</style>
""", unsafe_allow_html=True)

# =========================================================
# 4️⃣ LOAD DATA DESA
# =========================================================
DATA_URL = "https://drive.google.com/uc?id=1jmBB6Dv36aRnbDkj-cuZ154M0E3tzhOQ"
LOCAL_FILE = "desa1_riau.csv"

@st.cache_data(show_spinner=True)
def load_data():
    if not os.path.exists(LOCAL_FILE):
        gdown.download(DATA_URL, LOCAL_FILE, quiet=False, fuzzy=True)

    df = pd.read_csv(LOCAL_FILE)
    df.columns = [c.strip().upper() for c in df.columns]

    df = df.rename(columns={
        "WADMKD": "nama_desa",
        "NAMOBJ": "nama_desa",
        "DESA": "nama_desa",
        "WADMKK": "kabupaten"
    })

    df["geometry"] = df["WKT"].apply(lambda x: shapely.wkt.loads(x))
    df["lat"] = df.geometry.centroid.y
    df["lon"] = df.geometry.centroid.x

    return df.dropna().reset_index(drop=True)

# =========================================================
# 5️⃣ AMBIL DATA SATELIT (AUTO MUNDUR)
# =========================================================
def get_satellite_data(df):
    features = [
        ee.Feature(ee.Geometry.Point([r.lon, r.lat]), {"idx": i})
        for i, r in df.iterrows()
    ]
    fc = ee.FeatureCollection(features)

    now = datetime.utcnow()

    lst = ee.ImageCollection("MODIS/061/MOD11A1") \
        .filterDate(now - timedelta(days=8), now) \
        .select("LST_Day_1km").mean()

    ndvi = ee.ImageCollection("MODIS/061/MOD13Q1") \
        .filterDate(now - timedelta(days=32), now) \
        .select("NDVI").mean()

    rain = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
        .filterDate(now - timedelta(days=30), now) \
        .select("precipitation").sum()

    combined = lst.addBands(ndvi).addBands(rain).unmask(-9999)

    data = combined.reduceRegions(
        fc, ee.Reducer.first(), scale=1000
    ).getInfo()

    rows = []
    for f in data["features"]:
        p = f["properties"]
        rows.append({
            "idx": p["idx"],
            "LST": (p["LST_Day_1km"] * 0.02 - 273.15) if p["LST_Day_1km"] > 0 else np.nan,
            "NDVI": p["NDVI"] * 0.0001 if p["NDVI"] > 0 else np.nan,
            "Rain": p["precipitation"]
        })

    return df.merge(pd.DataFrame(rows), left_index=True, right_on="idx")

# =========================================================
# 6️⃣ HITUNG RISIKO KEBAKARAN
# =========================================================
def calculate_risk(df):
    norm_lst = ((df.LST - 25) / 15).clip(0, 1)
    norm_rain = (1 - df.Rain / 300).clip(0, 1)
    norm_ndvi = (1 - df.NDVI).clip(0, 1)

    df["prob_pct"] = ((0.4 * norm_lst + 0.4 * norm_rain + 0.2 * norm_ndvi) * 100).round(1)

    def level(p):
        if p > 70: return "TINGGI", [255, 0, 0]
        if p > 50: return "SEDANG", [255, 165, 0]
        return "RENDAH", [0, 128, 0]

    res = df.prob_pct.apply(level)
    df["level"] = [r[0] for r in res]
    df["color"] = [r[1] for r in res]

    return df

# =========================================================
# 7️⃣ MAIN APP
# =========================================================
def main():

    with st.sidebar:
        st.image("https://cdn-icons-png.flaticon.com/512/1041/1041891.png", width=60)
        st.success("🛰️ GEE ONLINE")
        if st.button("🔄 Refresh Data"):
            st.cache_data.clear()
            st.rerun()

    st.title("RIAU FIRE COMMAND CENTER")
    st.caption("Monitoring Kebakaran Hutan & Lahan Berbasis Satelit")

    df_base = load_data()
    df_sat = get_satellite_data(df_base)
    df = calculate_risk(df_sat)

    st.metric("🔥 Desa Risiko Tinggi", (df.level == "TINGGI").sum())

    geojson = {
        "type": "FeatureCollection",
        "features": [{
            "type": "Feature",
            "geometry": shapely.geometry.mapping(r.geometry),
            "properties": {
                "nama": r.nama_desa,
                "level": r.level,
                "prob": r.prob_pct,
                "color": r.color
            }
        } for _, r in df.iterrows()]
    }

    layer = pdk.Layer(
        "GeoJsonLayer",
        geojson,
        get_fill_color="properties.color",
        get_line_color=[0, 0, 0],
        line_width_min_pixels=2,
        pickable=True
    )

    st.pydeck_chart(pdk.Deck(
        layers=[layer],
        initial_view_state=pdk.ViewState(latitude=0.5, longitude=101.5, zoom=7),
        tooltip={"html": "<b>{nama}</b><br>Risiko: {level} ({prob}%)"}
    ))

    st.dataframe(
        df[["nama_desa", "kabupaten", "level", "prob_pct", "Rain", "LST", "NDVI"]],
        use_container_width=True,
        height=420
    )

if __name__ == "__main__":
    main()

